### note: change all of the paths to your correct path when loading in data

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os
import Levenshtein

In [2]:
pd.options.display.max_rows = 600
pd.options.display.max_columns = 85

In [3]:
home_tmp = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/mismatched_home.csv', header = None)
cdbg_tmp = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/mismatched_cdbg.csv', header = None)
mismatched_home = home_tmp.iloc[:,0]
mismatched_cdbg = cdbg_tmp.iloc[:,0]

In [4]:
mismatched_home.shape[0] + mismatched_cdbg.shape[0]

99

### need to drop untitled column in each

In [3]:
df_full_utilities = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/TotalUtilities.csv')

/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
s_addresses = pd.Series(df_full_utilities['Premise Address'].unique())

/anaconda3/envs/cds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/cds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
s_addresses[s_addresses=="529 COCHRAN AVE"]

Series([], dtype: object)

In [6]:
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
series_utilities = pd.Series(list2)

In [10]:
s_utilities = pd.Series(series_utilities.unique())

NameError: name 'series_utilities' is not defined

In [12]:
xlsx = pd.ExcelFile('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)

In [23]:
df_houses = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/TotalHome.csv')

### helper functions

In [8]:
def lev_list(address_list, address, i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [10]:
def space_fix_utilities(s_util):
    s_util_to_series = []
    for address in s_util:
        full_address = address.split()
        length = len(full_address)
        word = full_address[0] + "  "
        for i in range(1, length-1):
            word += full_address[i] + " "
        word += " " + full_address[length-1]
        word = word.strip()
        s_util_to_series.append(word)
    return pd.Series(s_util_to_series)

In [11]:
def construct_fixed_addresses(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if snm != "NA":
            word += snm + " "
        if stp != "NA":
            word += stp
        tmp_to_series.append(word.strip())
    return pd.Series(tmp_to_series)

In [12]:
def gen_list_none(address_list, mismatched, i):
    tmp_list = []
    for address in mismatched:
        if address == "UNKNOWN ADDRESS" or address == "SUPPRESSED ADDRESS":
            continue
        if lev_list(address_list, address, i).shape[0] == 0:
            tmp_list.append(address)
    return pd.Series(tmp_list)

### Read in more data

In [13]:
# s_utilities = space_fix_utilities(s_utilities)
df_parsehome = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_parsecdbg = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [14]:
s_parsehome = df_parsehome['Address']
s_parsecdbg = df_parsecdbg['Address']

In [15]:
def containsAPT(series):
    apt_list = []
    for address in series:
        if "APT" in address:
            apt_list.append(address)
    return pd.Series(apt_list)

In [16]:
def containsDUP(series):
    dup_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        for i in range(length - 1):
            word = full_address[i]
            if len(word) == 1:
                if not ((word == 'N') or (word == 'E')
                or (word == 'S') or (word == 'W')):
                    dup_list.append(address)
    return pd.Series(dup_list)

In [17]:
def clean(series):
    tmp_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        if any(char.isdigit() for char in full_address[length-1]):
            tmp_list.append(address)
        else:
            tmp_list.append("---")
    return pd.Series(tmp_list)

In [18]:
def splitter(series):
    apt_list = set(containsAPT(series))
    dup_list = set(containsDUP(series))
    sno_to_series = []
    snm_to_series = []
    stp_to_series = []
    apt_to_series = []
    dup_to_series = []

    for address in series:
        full_address = address.split()
        length = len(full_address)
        begin_index = 1
        end_index = length - 1
        #mutually exclusive apt and duplex
        if address == "UNKNOWN ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("UNKNOWN ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address == "SUPPRESSED ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("SUPPRESSED ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address in apt_list:
            apt_to_series.append(full_address[length-1])
            stp_to_series.append(full_address[length-3])
            end_index -= 2
        else:
            apt_to_series.append("NA")
            stp_to_series.append(full_address[length-1])
            
        if address in dup_list:
            dup_to_series.append(full_address[1])
            begin_index += 1
        else:
            dup_to_series.append("NA")
        sno_to_series.append(full_address[0])
        to_str = full_address[begin_index:end_index]
        word = ""
        for part in to_str:
            word += part + " "
        snm_to_series.append(word.strip())
    series_apt = pd.Series(apt_to_series)
    series_dup = pd.Series(dup_to_series)
    series_sno = pd.Series(sno_to_series)
    series_snm = pd.Series(snm_to_series)
    series_stp = pd.Series(stp_to_series)
    return (series_apt, series_dup, series_sno, series_snm, series_stp)

In [19]:
series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home = splitter(s_parsehome)
series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg = splitter(s_parsecdbg)

In [20]:
home = [series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home]
cdbg = [series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg]
for hom in home:
    print(hom.shape)
for cdb in cdbg:
    print(cdb.shape)

(286,)
(286,)
(286,)
(286,)
(286,)
(564,)
(564,)
(564,)
(564,)
(564,)


In [21]:
def checker(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if dup != "NA":
            word += dup + " "
        word += snm + " "
        if stp != "NA":
            word += stp + " "
        if apt != "NA":
            word += "APT " + apt
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)

In [22]:
s_parsehome[checker(home) != s_parsehome]

Series([], Name: Address, dtype: object)

In [23]:
df_all_addresses = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Processed-Data/all_addresses.csv')
df_all_addresses = df_all_addresses.drop('Unnamed: 0', axis = 1)
s_all_addresses = df_all_addresses['line1']

### Workflow

s_utilities is a list of all addresses that show up in the utilities data <br>
s_all_addresses is a list of all addresses in Albany <br>
df_original is a list of all housing projects <br>
lev_list takes in one of the two lists above, an address, and a threshold. <br>
If the address is within the threshold distance (string comparison) of an address in the two lists, it will print out.

In [24]:
mismatched_home

0             2620  S MADISON  ST
1               529  COCHRAN  AVE
2          2226  OLD DOMINION  DR
3             2701  DONCASTER  RD
4            250  BONNY VIEW  AVE
5            249  BONNY VIEW  AVE
6            248  BONNY VIEW  AVE
7            251  BONNY VIEW  AVE
8            253  BONNY VIEW  AVE
9            247  BONNY VIEW  AVE
10           254  BONNY VIEW  AVE
11           243  BONNY VIEW  AVE
12           245  BONNY VIEW  AVE
13           252  BONNY VIEW  AVE
14            2010  LAFAYETTE  CR
15              20  S JACKSON  ST
16               603  N HALEY  ST
17           1112  PEACHTREE  TER
18             2517  CARDINAL  DR
19            2006  LAFAYETTE  CR
20              818  WESTVIEW  DR
21              2501  REDWOOD  CR
22              933  RANDOLPH  DR
23            2201  CHAMPAGNE  DR
24              215  BROADWAY  ST
25              1003  HOLLAND  DR
26            2200  CHAMPAGNE  LN
27           1019  UNIVERSITY  DR
28            509  WESTOVER  BLVD
29            

In [25]:
mismatched_cdbg

0                         UNKNOWN ADDRESS
1                          400  EBONY  LN
2                    638  W HIGHLAND  AVE
3                         UNKNOWN ADDRESS
4                         UNKNOWN ADDRESS
5                         UNKNOWN ADDRESS
6                  230  S JACKSON ST  250
7                         UNKNOWN ADDRESS
8                         UNKNOWN ADDRESS
9                         UNKNOWN ADDRESS
10                       302  S CASON  ST
11                        UNKNOWN ADDRESS
12                     SUPPRESSED ADDRESS
13                     225  S JACKSON  ST
14                   1010  W LINCOLN  AVE
15    1401  E CAMPBELL ST/108 CARROLL  ST
16                     2572  CRESCENT  DR
17                     2572  CRESCENT  DR
18                       4835  IMPALA  LN
19                         507  EMILY  ST
20                    608  RESIDENCE  AVE
21                       614  ROADWAY  ST
22                       706  E TOWNE  RD
23                     SUPPRESSED 

### example (note the spacing difference between addresses in utilities and the full addr list)

In [14]:
lev_list(s_addresses, "529 COCHRAN AVE", 4)

0        829 CORN AVE
1        729 CORN AVE
2        509 CORN AVE
3        525 CORN AVE
4        524 CORN AVE
5        629 CORN AVE
6     520 COCHRAN AVE
7     606 COCHRAN AVE
8     518 COCHRAN AVE
9      527 COCHRAN ST
10     525 COCHRAN ST
11    521 COCHRAN AVE
12     519 COCHRAN ST
13    524 COCHRAN AVE
14    528 COCHRAN AVE
15      529 GOWAN AVE
16     529 COCHRAN ST
17     522 COCHRAN ST
18     526 COCHRAN ST
dtype: object

In [27]:
lev_list(s_all_addresses, "1010 W LINCOLN AVE", 4)

0      1310 W LINCOLN AVE
1      1308 W LINCOLN AVE
2      1306 W LINCOLN AVE
3      1304 W LINCOLN AVE
4      1302 W LINCOLN AVE
5      1300 W LINCOLN AVE
6      1230 W LINCOLN AVE
7      1228 W LINCOLN AVE
8      1226 W LINCOLN AVE
9      1224 W LINCOLN AVE
10     1224 W LINCOLN AVE
11     1222 W LINCOLN AVE
12     1220 W LINCOLN AVE
13     1218 W LINCOLN AVE
14     1218 W LINCOLN AVE
15     1216 W LINCOLN AVE
16     1212 W LINCOLN AVE
17     1208 W LINCOLN AVE
18     1206 W LINCOLN AVE
19     1204 W LINCOLN AVE
20     1202 W LINCOLN AVE
21     1200 W LINCOLN AVE
22     1126 W LINCOLN AVE
23     1124 W LINCOLN AVE
24     1122 W LINCOLN AVE
25     1120 W LINCOLN AVE
26     1118 W LINCOLN AVE
27     1116 W LINCOLN AVE
28     1114 W LINCOLN AVE
29     1112 W LINCOLN AVE
30     1110 W LINCOLN AVE
31     1108 W LINCOLN AVE
32     1106 W LINCOLN AVE
33     1104 W LINCOLN AVE
34     1400 W LINCOLN AVE
35     1402 W LINCOLN AVE
36     1404 W LINCOLN AVE
37     1512 W LINCOLN AVE
38     1510 

### So in the above example, 2620 S Madison St is found in the list of all addresses, but not found in the utilities data. Record this fact in the google doc. Then, look up the project type in the housing project data to also record what type of project it was. I'm looking to see if there is any trend with demolition or disposition projects not being found in utilities data.

### example (note the capitalization difference)

In [28]:
mismatched_cdbg

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
286,NaN,1008 East Residence,Albany,GA,NaN,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0
328,NaN,801 W Residence Ave,Albany,GA,NaN,2007,15,Public Service,572,Completed,CDBG,18572.87,18572.87,0.0
499,NaN,608 Residence Avenue,Albany,GA,NaN,2010,13,Demolition,941,Completed,CDBG,3341.25,3341.25,0.0
786,NaN,1505 E. Residence,Albany,GA,NaN,2016,5,Emergency Repairs,1470,Completed,CDBG,5884.47,5884.47,0.0
816,NaN,1329 E. Residence,Albany,GA,NaN,2017,20,Emergency Repairs,1518,Completed,CDBG,11896.43,11896.43,0.0


### Tests

In [29]:
mismatched_home

0                         UNKNOWN ADDRESS
1                          400  EBONY  LN
2                    638  W HIGHLAND  AVE
3                         UNKNOWN ADDRESS
4                         UNKNOWN ADDRESS
5                         UNKNOWN ADDRESS
6                  230  S JACKSON ST  250
7                         UNKNOWN ADDRESS
8                         UNKNOWN ADDRESS
9                         UNKNOWN ADDRESS
10                       302  S CASON  ST
11                        UNKNOWN ADDRESS
12                     SUPPRESSED ADDRESS
13                     225  S JACKSON  ST
14                   1010  W LINCOLN  AVE
15    1401  E CAMPBELL ST/108 CARROLL  ST
16                     2572  CRESCENT  DR
17                     2572  CRESCENT  DR
18                       4835  IMPALA  LN
19                         507  EMILY  ST
20                    608  RESIDENCE  AVE
21                       614  ROADWAY  ST
22                       706  E TOWNE  RD
23                     SUPPRESSED 

In [20]:
lev_list(s_utilities, "2312  GLENEAGLE  DR", 1)

NameError: name 'construct_fixed_addresses' is not defined

In [21]:
mismatched_home

0             2620  S MADISON  ST
1               529  COCHRAN  AVE
2          2226  OLD DOMINION  DR
3             2701  DONCASTER  RD
4            250  BONNY VIEW  AVE
5            249  BONNY VIEW  AVE
6            248  BONNY VIEW  AVE
7            251  BONNY VIEW  AVE
8            253  BONNY VIEW  AVE
9            247  BONNY VIEW  AVE
10           254  BONNY VIEW  AVE
11           243  BONNY VIEW  AVE
12           245  BONNY VIEW  AVE
13           252  BONNY VIEW  AVE
14            2010  LAFAYETTE  CR
15              20  S JACKSON  ST
16               603  N HALEY  ST
17           1112  PEACHTREE  TER
18             2517  CARDINAL  DR
19            2006  LAFAYETTE  CR
20              818  WESTVIEW  DR
21              2501  REDWOOD  CR
22              933  RANDOLPH  DR
23            2201  CHAMPAGNE  DR
24              215  BROADWAY  ST
25              1003  HOLLAND  DR
26            2200  CHAMPAGNE  LN
27           1019  UNIVERSITY  DR
28            509  WESTOVER  BLVD
29            

In [22]:
mismatched_cdbg

0                         UNKNOWN ADDRESS
1                          400  EBONY  LN
2                    638  W HIGHLAND  AVE
3                         UNKNOWN ADDRESS
4                         UNKNOWN ADDRESS
5                         UNKNOWN ADDRESS
6                  230  S JACKSON ST  250
7                         UNKNOWN ADDRESS
8                         UNKNOWN ADDRESS
9                         UNKNOWN ADDRESS
10                       302  S CASON  ST
11                        UNKNOWN ADDRESS
12                     SUPPRESSED ADDRESS
13                     225  S JACKSON  ST
14                   1010  W LINCOLN  AVE
15    1401  E CAMPBELL ST/108 CARROLL  ST
16                     2572  CRESCENT  DR
17                     2572  CRESCENT  DR
18                       4835  IMPALA  LN
19                         507  EMILY  ST
20                    608  RESIDENCE  AVE
21                       614  ROADWAY  ST
22                       706  E TOWNE  RD
23                     SUPPRESSED 

In [34]:
s_fixed_home

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
232,00238/00047/003,2312 Gleneagle Drive Apt. B,Albany,GA,"31.5532888, -84.2080964",2016,19,Tenant Based Rental Assistance,1476,Completed,HOME,1026.31,1026.31,0.00
233,00238/00047/001,2316-B Gleneagles Drive,Albany,GA,"31.5532685, -84.2086317",2016,19,Tenant Based Rental Assistance,1412,Completed,HOME,7609.94,7609.94,0.00
270,00238/00047/003,2312 Gleneagles Drive Apt. B,Albany,GA,"31.5532888, -84.2080964",2017,11,Tenant Based Rental Assistance,1494,Open,HOME,6689.00,3866.53,2822.47


In [37]:
df1_original.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 Joel Drive,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


[0        NA
 1        NA
 2        NA
 3        NA
 4        NA
 5        NA
 6        NA
 7        NA
 8        NA
 9        NA
 10       NA
 11       NA
 12       NA
 13       NA
 14       NA
 15       NA
 16       NA
 17       NA
 18       NA
 19       NA
 20       NA
 21       NA
 22       NA
 23       NA
 24       NA
 25       NA
 26       NA
 27       NA
 28       NA
 29       NA
 30       NA
 31       NA
 32       NA
 33       NA
 34       NA
 35       NA
 36       NA
 37       NA
 38       NA
 39       NA
 40       NA
 41       NA
 42       NA
 43       NA
 44       NA
 45       NA
 46       NA
 47       NA
 48       NA
 49       NA
 50       NA
 51       NA
 52       NA
 53       NA
 54       NA
 55       NA
 56       NA
 57       NA
 58       NA
 59       NA
 60       NA
 61       NA
 62       NA
 63       NA
 64       NA
 65       NA
 66       NA
 67       NA
 68       NA
 69       NA
 70       NA
 71       NA
 72       NA
 73       NA
 74       NA
 75       NA
 76       NA

In [58]:
def appendaddress(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        #if dup != "NA":
        #    word += dup
        word += snm + " "
        if stp != "NA":
            word += stp + " "
        #if apt != "NA":
        #    word += "APT " + apt
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)

In [63]:
def appendmulti(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        #if sno != "NA":
        #    word += sno + " "
        if dup != "NA":
            word += dup
        #word += snm + " "
        #if stp != "NA":
        #    word += stp + " "
        elif apt != "NA":
            word += apt
        else:
            word = "NA"
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)   

In [67]:
def appendapt(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        #if sno != "NA":
        #    word += sno + " "
        #if dup != "NA":
        #    word += dup
        #word += snm + " "
        #if stp != "NA":
        #    word += stp + " "
        if apt != "NA":
            word += "APT"
        else:
            word = "NA"
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)   

In [92]:
main_addr = appendaddress(home)
df1_original['Address'] = main_addr

In [93]:
apt_addr = appendapt(home)
df1_original['Multihome_Type'] = apt_addr

In [94]:
multi_addr = appendmulti(home)
df1_original['Multihome'] = multi_addr

In [95]:
main_addr_cdbg = appendaddress(cdbg)
multi_addr_cdbg = appendmulti(cdbg)
apt_addr_cdbg = appendapt(cdbg)

In [96]:
df2_original['Address'] = main_addr_cdbg
df2_original['Multihome_Type'] = apt_addr_cdbg
df2_original['Multihome'] = multi_addr_cdbg

In [66]:
multi_addr

0        NA
1        NA
2        NA
3        NA
4        NA
5        NA
6        NA
7        NA
8        NA
9        NA
10       NA
11       NA
12       NA
13       NA
14       NA
15       NA
16       NA
17       NA
18       NA
19       NA
20       NA
21       NA
22       NA
23       NA
24       NA
25       NA
26       NA
27       NA
28       NA
29       NA
30       NA
31       NA
32       NA
33       NA
34       NA
35       NA
36       NA
37       NA
38       NA
39       NA
40       NA
41       NA
42       NA
43       NA
44       NA
45       NA
46       NA
47       NA
48       NA
49       NA
50       NA
51       NA
52       NA
53       NA
54       NA
55       NA
56       NA
57       NA
58       NA
59       NA
60       NA
61       NA
62       NA
63       NA
64       NA
65       NA
66       NA
67       NA
68       NA
69       NA
70       NA
71       NA
72       NA
73       NA
74       NA
75       NA
76       NA
77       NA
78       NA
79       NA
80        A
81        A
82       NA
83  

In [52]:
df1_original

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.00
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.00
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.00
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.00
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.00
5,0000K/00027/008,1519 MALONE DR,Albany,GA,"31.6013883, -84.1132275",2007,6,Homeowner Rehabilitation,631,Completed,HOME,61252.07,61252.07,0.00
6,0000U/00003/010,719 ANDOVER LN,Albany,GA,"31.5851881, -84.1350519",2007,6,Homeowner Rehabilitation,621,Completed,HOME,29927.62,29927.62,0.00
7,0000Q/00026/007,305 S SHADOWLAWN DR,Albany,GA,"31.5739048, -84.1933113",2007,6,Homeowner Rehabilitation,599,Completed,HOME,28778.79,28778.79,0.00
8,000HH/00062/042,1662 SUNNY LN,Albany,GA,"31.5548264, -84.1631739",2007,6,Homeowner Rehabilitation,559,Completed,HOME,37189.39,37189.39,0.00
9,00001/00031/24D,511 MOULTRIE RD,Albany,GA,"31.557129, -84.1338254",2007,6,Homeowner Rehabilitation,611,Completed,HOME,30159.70,30159.70,0.00


In [69]:
apt_addr

0       NA
1       NA
2       NA
3       NA
4       NA
5       NA
6       NA
7       NA
8       NA
9       NA
10      NA
11      NA
12      NA
13      NA
14      NA
15      NA
16      NA
17      NA
18      NA
19      NA
20      NA
21      NA
22      NA
23      NA
24      NA
25      NA
26      NA
27      NA
28      NA
29      NA
30      NA
31      NA
32      NA
33      NA
34      NA
35      NA
36      NA
37      NA
38      NA
39      NA
40      NA
41      NA
42      NA
43      NA
44      NA
45      NA
46      NA
47      NA
48      NA
49      NA
50      NA
51      NA
52      NA
53      NA
54      NA
55      NA
56      NA
57      NA
58      NA
59      NA
60      NA
61      NA
62      NA
63      NA
64      NA
65      NA
66      NA
67      NA
68      NA
69      NA
70      NA
71      NA
72      NA
73      NA
74      NA
75      NA
76      NA
77      NA
78      NA
79      NA
80      NA
81      NA
82      NA
83      NA
84      NA
85      NA
86      NA
87      NA
88      NA
89      NA
90      NA

In [97]:
df1_original

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.00,NA,NA
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.00,NA,NA
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.00,NA,NA
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.00,NA,NA
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.00,NA,NA
5,0000K/00027/008,1519 MALONE DR,Albany,GA,"31.6013883, -84.1132275",2007,6,Homeowner Rehabilitation,631,Completed,HOME,61252.07,61252.07,0.00,NA,NA
6,0000U/00003/010,719 ANDOVER LN,Albany,GA,"31.5851881, -84.1350519",2007,6,Homeowner Rehabilitation,621,Completed,HOME,29927.62,29927.62,0.00,NA,NA
7,0000Q/00026/007,305 S SHADOWLAWN DR,Albany,GA,"31.5739048, -84.1933113",2007,6,Homeowner Rehabilitation,599,Completed,HOME,28778.79,28778.79,0.00,NA,NA
8,000HH/00062/042,1662 SUNNY LN,Albany,GA,"31.5548264, -84.1631739",2007,6,Homeowner Rehabilitation,559,Completed,HOME,37189.39,37189.39,0.00,NA,NA
9,00001/00031/24D,511 MOULTRIE RD,Albany,GA,"31.557129, -84.1338254",2007,6,Homeowner Rehabilitation,611,Completed,HOME,30159.70,30159.70,0.00,NA,NA


In [98]:
df2_original

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,NaN,1008 E RESIDENCE AVE,Albany,GA,NaN,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.00,NA,NA
1,NaN,1100 SWIFT ST,Albany,GA,NaN,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.00,NA,NA
2,NaN,1117 W WADDELL AVE,Albany,GA,NaN,2007,5,Energy Efficiency,607,Completed,CDBG,3977.52,3977.52,0.00,NA,NA
3,NaN,1223 E BROAD AVE,Albany,GA,NaN,2007,7,Emergency Repairs,575,Completed,CDBG,4776.14,4776.14,0.00,NA,NA
4,NaN,1316 E 4TH AVE,Albany,GA,NaN,2007,7,Emergency Repairs,613,Completed,CDBG,6934.40,6934.40,0.00,NA,NA
5,NaN,1419 W HIGHLAND AVE,Albany,GA,NaN,2007,5,Energy Efficiency,603,Completed,CDBG,3985.52,3985.52,0.00,NA,NA
6,NaN,1607 AVALON AVE,Albany,GA,NaN,2007,7,Emergency Repairs,587,Completed,CDBG,4818.22,4818.22,0.00,NA,NA
7,NaN,1662 SUNNY LN,Albany,GA,NaN,2007,5,Energy Efficiency,616,Completed,CDBG,3402.52,3402.52,0.00,NA,NA
8,NaN,1706 S JEFFERSON ST,Albany,GA,NaN,2007,4,Economic Development,628,Completed,CDBG,2156.32,2156.32,0.00,NA,NA
9,NaN,1712 W HIGHLAND AVE,Albany,GA,NaN,2007,5,Energy Efficiency,605,Completed,CDBG,4802.52,4802.52,0.00,NA,NA


In [15]:
df = pd.concat([df1_original, df2_original], ignore_index = True)

In [103]:
df.to_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/TotalHome.csv', index = False)

In [25]:
s_house_addresses = pd.Series(df_houses['Address'].unique())

In [26]:
s_addresses

0                      212 ACORN ST
1                  2101 LEONARD AVE
2                  2103 LEONARD AVE
3                 636 JEFFERIES AVE
4                  2105 LEONARD AVE
5                      201 ELSOM ST
6                      203 ELSOM ST
7                      205 ELSOM ST
8                      207 ELSOM ST
9                      209 ELSOM ST
10                  2214 HUBERT AVE
11                     301 ELSOM ST
12                     303 ELSOM ST
13                640 JEFFERIES AVE
14                     305 ELSOM ST
15                     309 ELSOM ST
16                     313 ELSOM ST
17                     314 ELSOM ST
18                     306 ELSOM ST
19                     300 ELSOM ST
20                   1504 NEWTON RD
21                  2305 HUBERT AVE
22                  2309 HUBERT AVE
23                 2200 LEONARD AVE
24                  109 N MOCK N RD
25                    107 N MOCK RD
26                   1506 NEWTON RD
27                   2100 CL

In [30]:
tmp = s_house_addresses[~s_house_addresses.isin(s_addresses)]

In [32]:
tmp.to_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/mismatched_house_addresses.csv', header = False)

In [24]:
df_houses.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance,Multihome_Type,Multihome
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0,NaN,NaN
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0,NaN,NaN
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0,NaN,NaN
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0,NaN,NaN
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0,NaN,NaN
